## Deep Learning Model Optimization

#### Dependencies Required

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier 

#### Import CSV file

In [2]:
import pandas as pd 
df = pd.read_csv("charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
df = df.drop(["EIN"],1)

In [4]:
df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

#### Examine for Binning

In [5]:
name_counts = df['NAME'].value_counts()
name_counts[name_counts>5]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
TEXAS FEDERATION OF WOMEN CLUBS                       6
NATIONAL CHARITY LEAGUE INC                           6
CBMC INC                                              6
THE READING FOUNDATION                                6
FULBRIGHT ASSOCIATION INC                             6
Name: NAME, Length: 354, dtype: int64

#### Replace Names

In [6]:
names_to_replace = list(name_counts[name_counts <= 5].index)

# Loop to Replace Names
for app in names_to_replace:
    df['NAME'] = df['NAME'].replace(app,"Other")
    
# Verification
df['NAME'].value_counts()

Other                                                         20043
PARENT BOOSTER USA INC                                         1260
TOPS CLUB INC                                                   765
UNITED STATES BOWLING CONGRESS INC                              700
WASHINGTON STATE UNIVERSITY                                     492
                                                              ...  
TEXAS FEDERATION OF WOMEN CLUBS                                   6
HABITAT FOR HUMANITY INTERNATIONAL                                6
MONTANA CONGRESS OF PARENTS TEACHERS STUDENTS                     6
DEPARTMENT OF OREGON LADIES AUXILLARY TO THE VFW OF THE US        6
HANDS FOR LIFE                                                    6
Name: NAME, Length: 355, dtype: int64

#### Application Types

In [7]:
application_counts = df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

#### Replace Application Types

In [8]:
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Loop for Replacing Application Types
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# verification
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

#### Classification Binning

In [9]:
# Look at CLASSIFICATION value counts for binning
class_counts = df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C4200        1
C1248        1
C2561        1
C1236        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
classes_to_replace = list(class_counts[class_counts < 1000].index)

# Loop to Replace Classification
for cls in classes_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Verification
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

#### Create Variable List & Create Dummies

In [12]:
# Generate our categorical variable lists
application_cat = df.dtypes[df.dtypes == "object"].index.tolist()

In [13]:
application_with_dummies_df = pd.get_dummies(df)

#### Split Preprocessed Data

In [14]:
X = application_with_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_with_dummies_df["IS_SUCCESSFUL"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

#### Scale Data

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Define Neural Net

In [16]:
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               39900     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43,251
Trainable params: 43,251
Non-trainable params: 0
_________________________________________________________________


#### Compile Model

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#### Train Model

In [18]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4957 - accuracy: 0.7752
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4442 - accuracy: 0.7929
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4345 - accuracy: 0.7954
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4300 - accuracy: 0.7956
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4276 - accuracy: 0.7975
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4253 - accuracy: 0.7972
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4242 - accuracy: 0.7975
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4225 - accuracy: 0.7997
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4214 - accuracy: 0.7997
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4205 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.4021 - accuracy: 0.8082
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4020 - accuracy: 0.8088
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4018 - accuracy: 0.8086
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4023 - accuracy: 0.8082
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4024 - accuracy: 0.8078
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8078
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4017 - accuracy: 0.8081
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4019 - accuracy: 0.8085
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4023 - accuracy: 0.8080
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4027 - accuracy: 

#### Model Evaluation & Accuracy

In [19]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4463 - accuracy: 0.7893 - 357ms/epoch - 1ms/step
Loss: 0.44630560278892517, Accuracy: 0.7892711162567139


#### Export Model to .h5 file

In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

#### Random Forest Classification Model, Fitting, Evaluation

In [21]:
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

rf_model = rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest model accuracy: 0.776


## Data Preprocessing

#### What variable(s) are considered the target(s) for your model?

    "Is-Successful" Column

#### What variable(s) are considered to be the features for your model?

    Name, Application, Type, Afflication, Calssification, Use_Case, Organization, Income_Amt, Special_Considerations, Status, & Ask-Amt

#### What variable(s) are neither targets nor features, and should be removed from the input data?

        EIN was dropped. Sepecial_Considerations and Status both were not siginficant to the model

## Compiling, Training, & Evaluating the Model

#### How many neurons, layers, and activation functions did you select for your neural network model, and why?

    Three hidden layers were required to meet the required 75% accuracy. Relu, Sigmoid, & Sigmoid activation functions also helped increase the accuracy.

#### Were you able to achieve the target model perforWere you able to achieve the target model performance?

    Yes the model met the required minmum accuracy of 75%

#### What steps did you take to try and increase model performance?

    3 Layers and (Relu, Sigmoid, Sigmoid) changes optimized the performance to meet the 75% accuracry requirement

## Summary

    By optmizing the Neural Network the accuracy was increased to 78%. Howevery by using a Random Forest Classification Model the accuracry was 77.6%. Although the accuracy on the NN model was higher, it took more computing power and time to train. Therefore a Random Forest Classification Model is prefered or future development. 